In [184]:
# import agent

In [185]:
# # Input
# initial_input = {"messages": [{"role": "user", "content": "Mi dai l'intesità di precipitazione?"}] }

# # Thread
# thread = {"configurable": {"thread_id": "4"}}

# # Run the graph until the first interruption
# for event in agent.graph.stream(initial_input, thread, stream_mode="updates"):
#     print('evt: --- ', event)
#     print("\n")

In [186]:
# # Input
# user_input = {"messages": [{"role": "user", "content": "A milano il 15 sett. 2023"}] }

# # Run the graph until the first interruption
# for event in agent.graph.stream(user_input, thread, stream_mode="updates"):
#     print('evt: --- ', event)
#     print("\n")

In [187]:
# from geopy.geocoders import Nominatim

# def get_bounding_box(location_name):
#     geolocator = Nominatim(user_agent="bounding_box_finder")
#     location = geolocator.geocode(location_name, exactly_one=True)

#     if location and hasattr(location, 'raw'):
#         bounding_box = location.raw.get('boundingbox', None)
#         if bounding_box:
#             min_lat, max_lat, min_lon, max_lon = map(float, bounding_box)
#             return [min_lon, min_lat, max_lon, max_lat]
    
#     return None

# get_bounding_box('Georgia, GE')

In [1]:
import json
from agent.utils import _llm

In [14]:
ipynb_json = json.load(open(r'C:\Users\tommaso\AppData\Local\Temp\icisk-chat\spi_[8.4978518, 44.6799091, 11.4276477, 46.6353523]_notebook.ipynb', 'r'))
cells = ipynb_json.get('cells', [])
notebook_source_code = '\n'.join(['\n'.join(cell['source']) for cell in cells])

sys_message = f"""
    Sei un assistente di programmazione che aiuta gli utenti a scrivere codice python.
    Ricorda che il codice è relativo ad un analisi di dati geospaziali, nel caso di richieste relative a mappe, utilizza la libreria folium oppure cartopy.
    Ti è stato chiesto di scrivere del codice python che aggiunga una mappa che visualizzi la media nel tempo dei valori di SPI di ogni cella lat-lon.
    Il codice prodotto deve essere aggiunto a questo codice già esistente.
    
    {notebook_source_code}
    
    Rispondi esclusivamente con righe di codice python che sia integrabile con il codice già esistente. Deve utilizzare le variabili opprtune già definite nel codice.
    Non allegare altro testo.
    Non produrre codice ulteriore rispetto a quello necessario per soddisfare le richieste dichiarate nel parametro code_lines.
"""

code_lines = _llm.invoke([{"type": "system", "content": sys_message}])

In [15]:
code_lines

AIMessage(content="```python\nimport folium\nfrom folium import plugins\n\n# Calcola la media nel tempo dei valori di SPI\nmean_spi = spi_dataset.spi.mean(dim='time')\n\n# Crea una mappa centrata sulla regione di interesse\nmappa_spi = folium.Map(location=[(region[1]+region[3])/2, (region[0]+region[2])/2], zoom_start=6)\n\n# Aggiungi i valori di SPI sulla mappa\nfor lat in mean_spi.lat.values:\n    for lon in mean_spi.lon.values:\n        value = mean_spi.sel(lat=lat, lon=lon).values\n        folium.CircleMarker(location=[lat, lon], radius=5,\n                            color='blue' if value >= 0 else 'red',\n                            fill=True, fill_opacity=0.6,\n                            popup=f'SPI: {value:.2f}').add_to(mappa_spi)\n\n# Mostra la mappa\nmappa_spi\n```", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 203, 'prompt_tokens': 2806, 'total_tokens': 3009, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio

In [16]:
for l in code_lines.content.split('\n'):
    print(l)

```python
import folium
from folium import plugins

# Calcola la media nel tempo dei valori di SPI
mean_spi = spi_dataset.spi.mean(dim='time')

# Crea una mappa centrata sulla regione di interesse
mappa_spi = folium.Map(location=[(region[1]+region[3])/2, (region[0]+region[2])/2], zoom_start=6)

# Aggiungi i valori di SPI sulla mappa
for lat in mean_spi.lat.values:
    for lon in mean_spi.lon.values:
        value = mean_spi.sel(lat=lat, lon=lon).values
        folium.CircleMarker(location=[lat, lon], radius=5,
                            color='blue' if value >= 0 else 'red',
                            fill=True, fill_opacity=0.6,
                            popup=f'SPI: {value:.2f}').add_to(mappa_spi)

# Mostra la mappa
mappa_spi
```


In [17]:
import re

def clean_code_block(text):
    """Rimuove eventuali delimitatori di codice markdown (``` e ```python) da un blocco di codice."""
    return re.sub(r"^```(?:python)?\n|\n```$", "", text, flags=re.MULTILINE)

# Esempio di utilizzo
code_block = """```python
import folium
from folium import plugins

# Calcola la media nel tempo dei valori di SPI
mean_spi = spi_dataset.spi.mean(dim='time')

# Crea una mappa centrata sulla regione di interesse
mappa_spi = folium.Map(location=[(region[1]+region[3])/2, (region[0]+region[2])/2], zoom_start=6)

# Aggiungi i valori di SPI sulla mappa
for lat in mean_spi.lat.values:
    for lon in mean_spi.lon.values:
        value = mean_spi.sel(lat=lat, lon=lon).values
        folium.CircleMarker(location=[lat, lon], radius=5,
                            color='blue' if value >= 0 else 'red',
                            fill=True, fill_opacity=0.6,
                            popup=f'SPI: {value:.2f}').add_to(mappa_spi)

# Mostra la mappa
mappa_spi
```"""

cleaned_code = clean_code_block(code_block)
print(cleaned_code)


import folium
from folium import plugins

# Calcola la media nel tempo dei valori di SPI
mean_spi = spi_dataset.spi.mean(dim='time')

# Crea una mappa centrata sulla regione di interesse
mappa_spi = folium.Map(location=[(region[1]+region[3])/2, (region[0]+region[2])/2], zoom_start=6)

# Aggiungi i valori di SPI sulla mappa
for lat in mean_spi.lat.values:
    for lon in mean_spi.lon.values:
        value = mean_spi.sel(lat=lat, lon=lon).values
        folium.CircleMarker(location=[lat, lon], radius=5,
                            color='blue' if value >= 0 else 'red',
                            fill=True, fill_opacity=0.6,
                            popup=f'SPI: {value:.2f}').add_to(mappa_spi)

# Mostra la mappa
mappa_spi


In [233]:
state = {
  "values": {
    "messages": [
      {
        "content": "SPI per la lombardia",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": None,
        "id": "2b6e218e-b24a-4bb3-81e7-87b8faeed25c",
        "example": False
      },
      {
        "content": "\n            L'utente a rihiesto di eseguire il tool spi_notebook_creation con  questi argomenti:\n            - region: Lombardia\n            - reference_period: NULL\n            - period_of_interest: NULL\n            Non ha specificato il periodo di riferimento. Chiedigli se vuole usare l'intervallo 1980-2010 di deafult oppure preferisce specificarlo.\n        ",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "system",
        "name": None,
        "id": "4e1e850c-a399-4642-90d8-098b4b6e2bdc"
      },
      {
        "content": "Per calcolare lo Standardized Precipitation Index (SPI) per la Lombardia, possiamo utilizzare l'intervallo di riferimento predefinito 1981-2010, oppure posso chiedere all'utente di specificare un periodo diverso. Cosa preferisci?",
        "additional_kwargs": {},
        "response_metadata": {
          "finish_reason": "stop",
          "model_name": "gpt-3.5-turbo-0125"
        },
        "type": "ai",
        "name": None,
        "id": "run-416b31ae-fd78-436c-99ab-31feed64dc4d",
        "example": False,
        "tool_calls": [],
        "invalid_tool_calls": [],
        "usage_metadata": None
      },
      {
        "content": "ok quello di default",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": None,
        "id": "6c0aae96-aef7-462f-bcf9-78b2a7a1918d",
        "example": False
      },
      {
        "content": "\n            L'utente a rihiesto di eseguire il tool spi_notebook_creation con  questi argomenti:\n            - region: Lombardia\n            - reference_period: [1981, 2010]\n            - period_of_interest: NULL\n            Non ha specificato il periodo di interesse. Chiedigli se vuole usare l'intervallo di default (il mese precedente alla data corrente) oppure preferisce specificarlo.\n        ",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "system",
        "name": None,
        "id": "353f9bf2-8dae-4332-bda7-69782134d950"
      },
      {
        "content": "Per calcolare lo Standardized Precipitation Index (SPI) per la Lombardia utilizzando l'intervallo di riferimento 1981-2010, possiamo procedere con l'analisi per il periodo di interesse predefinito, che è il mese precedente alla data corrente. Se desideri utilizzare un periodo di interesse diverso, per favore specifica le date di inizio e fine del periodo desiderato.",
        "additional_kwargs": {},
        "response_metadata": {
          "finish_reason": "stop",
          "model_name": "gpt-3.5-turbo-0125"
        },
        "type": "ai",
        "name": None,
        "id": "run-0a90d810-39df-4e21-9872-0735012e749f",
        "example": False,
        "tool_calls": [],
        "invalid_tool_calls": [],
        "usage_metadata": None
      },
      {
        "content": "faciamo il mese scorso",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": None,
        "id": "402144fc-696f-4e95-b89f-ab814348f401",
        "example": False
      },
      {
        "content": "\n                    L'utente a rihiesto di eseguire il tool spi_notebook_creation con  questi argomenti:\n                    - region: [8.4978518, 44.6799091, 11.4276477, 46.6353523]\n                    - reference_period: [1981, 2010]\n                    - period_of_interest: ['2023-10', '2023-10']\n                    Per l'argomento region è stata ricavata questa bounding box: [8.4978518, 44.6799091, 11.4276477, 46.6353523]. Chiedi all'utente se è corretta oppure se vuole modificarla.\n                ",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "system",
        "name": None,
        "id": "d80199e0-16ae-4308-ad2e-905b3fb03479"
      },
      {
        "content": "La bounding box ricavata per la regione della Lombardia è la seguente: [8.4978518, 44.6799091, 11.4276477, 46.6353523]. È corretta o desideri apportare delle modifiche?",
        "additional_kwargs": {},
        "response_metadata": {
          "finish_reason": "stop",
          "model_name": "gpt-3.5-turbo-0125"
        },
        "type": "ai",
        "name": None,
        "id": "run-ca136595-e11d-4efe-91e4-05fc1f0ca286",
        "example": False,
        "tool_calls": [],
        "invalid_tool_calls": [],
        "usage_metadata": None
      },
      {
        "content": "va bene quella!",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": None,
        "id": "68b13b58-947c-42b6-9969-2319114338a3",
        "example": False
      }
    #   {
    #     "content": "",
    #     "additional_kwargs": {
    #       "tool_calls": [
    #         {
    #           "index": 0,
    #           "id": "call_1NXmr6nR8PMElNqMO5VSls3Z",
    #           "function": {
    #             "arguments": "{\"region\":[8.4978518,44.6799091,11.4276477,46.6353523],\"reference_period\":[1981,2010],\"period_of_interest\":[\"2023-10\",\"2023-10\"]}",
    #             "name": "spi_notebook_creation"
    #           },
    #           "type": "function"
    #         }
    #       ]
    #     },
    #     "response_metadata": {
    #       "finish_reason": "tool_calls",
    #       "model_name": "gpt-3.5-turbo-0125"
    #     },
    #     "type": "ai",
    #     "name": None,
    #     "id": "run-c777286d-7494-460d-abf2-73701f9bdbc4",
    #     "example": False,
    #     "tool_calls": [
    #       {
    #         "name": "spi_notebook_creation",
    #         "args": {
    #           "region": [
    #             8.4978518,
    #             44.6799091,
    #             11.4276477,
    #             46.6353523
    #           ],
    #           "reference_period": [
    #             1981,
    #             2010
    #           ],
    #           "period_of_interest": [
    #             "2023-10",
    #             "2023-10"
    #           ]
    #         },
    #         "id": "call_1NXmr6nR8PMElNqMO5VSls3Z",
    #         "type": "tool_call"
    #       }
    #     ],
    #     "invalid_tool_calls": [],
    #     "usage_metadata": None
    #   },
    #   {
    #     "content": "C:\\Users\\tommaso\\AppData\\Local\\Temp\\icisk-chat\\spi_[8.4978518, 44.6799091, 11.4276477, 46.6353523]_notebook.ipynb",
    #     "additional_kwargs": {},
    #     "response_metadata": {},
    #     "type": "tool",
    #     "name": "spi_notebook_creation",
    #     "id": "894a84ae-d9b2-4c9b-acb7-444274a7cd12",
    #     "tool_call_id": "call_1NXmr6nR8PMElNqMO5VSls3Z",
    #     "artifact": None,
    #     "status": "success"
    #   },
    #   {
    #     "content": "Ho creato con successo un nuovo notebook Jupyter per calcolare lo Standardized Precipitation Index (SPI) per la Lombardia con l'intervallo di riferimento 1981-2010 e il mese di interesse ottobre 2023. Puoi scaricare il notebook dal seguente percorso:\n\n[saved notebook](C:\\Users\\tommaso\\AppData\\Local\\Temp\\icisk-chat\\spi_[8.4978518, 44.6799091, 11.4276477, 46.6353523]_notebook.ipynb)",
    #     "additional_kwargs": {},
    #     "response_metadata": {
    #       "finish_reason": "stop",
    #       "model_name": "gpt-3.5-turbo-0125"
    #     },
    #     "type": "ai",
    #     "name": None,
    #     "id": "run-65126b67-0c85-467a-bb29-44c7b50d05fe",
    #     "example": False,
    #     "tool_calls": [],
    #     "invalid_tool_calls": [],
    #     "usage_metadata": None
    #   }
    ]
  },
  "next": [],
  "tasks": [],
  "metadata": {
    "x-auth-scheme": "langsmith",
    "langgraph_auth_user": None,
    "langgraph_auth_user_id": "",
    "langgraph_auth_permissions": [],
    "thread_ts": "1f004cb6-802f-683b-800d-5d93bb40fa32",
    "graph_id": "agent",
    "assistant_id": "fe096781-5601-53d2-b2f6-0d3403f7e9ca",
    "user_id": "",
    "created_by": "system",
    "from_studio": True,
    "run_attempt": 1,
    "langgraph_version": "0.3.10",
    "langgraph_plan": "developer",
    "langgraph_host": "self-hosted",
    "thread_id": "41bd84db-92c5-49d8-bcd8-38bde2590c15",
    "checkpoint_id": "1f004cb6-802f-683b-800d-5d93bb40fa32",
    "run_id": "1f004cb7-c394-6ef4-acac-d6d7ad6424b6",
    "source": "loop",
    "writes": {
      "chatbot": {
        "messages": [
          {
            "content": "Ho creato con successo un nuovo notebook Jupyter per calcolare lo Standardized Precipitation Index (SPI) per la Lombardia con l'intervallo di riferimento 1981-2010 e il mese di interesse ottobre 2023. Puoi scaricare il notebook dal seguente percorso:\n\n[saved notebook](C:\\Users\\tommaso\\AppData\\Local\\Temp\\icisk-chat\\spi_[8.4978518, 44.6799091, 11.4276477, 46.6353523]_notebook.ipynb)",
            "additional_kwargs": {},
            "response_metadata": {
              "finish_reason": "stop",
              "model_name": "gpt-3.5-turbo-0125"
            },
            "type": "ai",
            "name": None,
            "id": "run-65126b67-0c85-467a-bb29-44c7b50d05fe",
            "example": False,
            "tool_calls": [],
            "invalid_tool_calls": [],
            "usage_metadata": None
          }
        ]
      }
    },
    "step": 19,
    "parents": {}
  },
  "created_at": "2025-03-19T14:07:29.602452+00:00",
  "checkpoint": {
    "checkpoint_id": "1f004cb7-e855-63ca-8013-1c67426aac11",
    "thread_id": "41bd84db-92c5-49d8-bcd8-38bde2590c15",
    "checkpoint_ns": ""
  },
  "parent_checkpoint": {
    "checkpoint_id": "1f004cb7-d7b0-60d4-8012-6b96367ffd7e",
    "thread_id": "41bd84db-92c5-49d8-bcd8-38bde2590c15",
    "checkpoint_ns": ""
  },
  "checkpoint_id": "1f004cb7-e855-63ca-8013-1c67426aac11",
  "parent_checkpoint_id": "1f004cb7-d7b0-60d4-8012-6b96367ffd7e"
}

msgs = state['values']['messages']

In [234]:
need_confirm_msg = """
    L'utente a rihiesto di eseguire il tool spi_notebook_creation con questi argomenti:
    - region: [8.4978518,44.6799091,11.4276477,46.6353523]
    - reference_period: [1981,2010]
    - period_of_interest: ["2023-10","2023-10"]
    Non eseguire la chimata. Chiedi all'utente se i parametri sono corretti.
"""

msgs.append({'type': 'system', 'content': need_confirm_msg})

asked_confirm = _llm.invoke(msgs)
asked_confirm.content

''

In [221]:
confirm_msg = """
   No aspetta
"""
msgs.append({'type': 'human', 'content': confirm_msg})

confirm = _llm.invoke(msgs)
confirm.content

''

In [222]:
confirm

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iSnKyTZtoZasHUFUYYprign3', 'function': {'arguments': '{"region":[8.4978518,44.6799091,11.4276477,46.6353523],"reference_period":[1981,2010],"period_of_interest":["2023-10","2023-10"]}', 'name': 'spi_notebook_creation'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 1102, 'total_tokens': 1164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6b11f1e7-74c8-4241-8897-3ab6a9a0a6aa-0', tool_calls=[{'name': 'spi_notebook_creation', 'args': {'region': [8.4978518, 44.6799091, 11.4276477, 46.6353523], 'reference_period': [1981, 2010], 'period_of_interest': ['2023-10', '2023-10']}, 'i